# Portfolio Tracker
Run can your own portfolio tracker locally. 

## Notes
* Dividend is not reinvested, it is accumulated on the balance
* All transaction dates must fall on trading day (calendar)

In [1]:
import os
import pytz
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date, datetime, timedelta
import hashlib

In [2]:
# Configurations ###
transactions_sheet = ('transactions.xlsx', 'A')
benchmark_tickers = ['^GSPC', '^DJI', '^IXIC']
currency_ticker = 'USDGBP=X'  # also used for tracking cash, use USD___=X for any other currency
#
logs = 2  # 0 = none, 1 = all, 2 = minimum

In [3]:
benchmark_ticker = benchmark_tickers[0]
# read in the transactions from excel
transactions = pd.read_excel(transactions_sheet[0], sheet_name=transactions_sheet[1], parse_dates=[0])
tickers = transactions['symbol'].dropna().unique().tolist()  # collect tickers
for idx, t in transactions.iterrows():
    if not pd.isna(t['cash']) and pd.isna(t['symbol']):  # no symbol, there is a cash value
        transactions.at[idx] = [t['date'], 'BUY', currency_ticker, 0, 0, 0, t['cash']]
    else:  # otherwise make sure the cash is 0
        transactions.at[idx, 'cash'] = 0
if logs > 1: transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
date          5 non-null datetime64[ns]
tx            5 non-null object
symbol        5 non-null object
quantity      5 non-null float64
price         5 non-null float64
commission    5 non-null float64
cash          5 non-null float64
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 408.0+ bytes


In [4]:
# calculate the start and end dates for the analysis in UTC
# start date = date of the very first transaction
# end date = yesterday
start_date = transactions['date'].min().replace(tzinfo=pytz.UTC)
end_date = datetime.combine(date.today(), datetime.min.time()).replace(tzinfo=pytz.UTC) + timedelta(days=-1)
if logs > 0: print(f'start date = {start_date}\nend date = {end_date}')

start date = 2015-01-05 00:00:00+00:00
end date = 2019-07-22 00:00:00+00:00


In [5]:
# create transactions for benchmarks - buying 1 unit of index on start date
for t in benchmark_tickers:
    transactions = transactions.append({'date': start_date, 'tx': 'BUY', 'symbol': t,
                                        'quantity': 1, 'price': np.nan, 'commission': 0, 'cash': 0},
                                       ignore_index=True)

In [6]:
# get EoD data from Yahoo or file (cached)
# EoD data is cached in file for faster subsequent runs
# calculate hash : start_date, end_date, all the tickers
hash_string = (start_date.isoformat() + end_date.isoformat() + ','.join(tickers)).encode('utf8')
h = hashlib.md5(hash_string).hexdigest()
filename = f'{h}.dat'
if os.path.isfile(filename):
    if logs > 0: print(f'File {filename} exist, loading EoD data from file')
    eod_data = pd.read_pickle(filename)
else:
    if logs > 0: print(f'File {filename} does not exist, loading EoD data from Yahoo')
    # load the benchmark ticker first to get the index (calendar)
    # NOTE: the benchmark ticker is loaded twice - it's OK
    benchmark_eod = yf.Ticker(benchmark_ticker).history(start=start_date, end=end_date)
    # load end of day data from Yahoo using yfinance - https://github.com/ranaroussi/yfinance
    def get_eod(tickers, startdate, enddate):

        def get_data(ticker):
            if logs > 1: print(f'Loading {ticker} from Yahoo')
            data = yf.Ticker(ticker).history(start=startdate, end=enddate, auto_adjust=True, actions=True)
            data.drop(['Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)
            return data.reindex(benchmark_eod.index, fill_value=0)  # reindex to stretch full date range

        data_map = map(get_data, tickers)
        return (pd.concat(data_map, keys=tickers, names=['symbol', 'date']))

    eod_data = get_eod(tickers + [currency_ticker] + benchmark_tickers, start_date, end_date)
    # cache EoD data in file
    if logs > 1: print(f'Writing EoD data to file {filename}')
    eod_data.to_pickle(filename)

File a3664bf82a7b89a00f9e973ec3c9a022.dat exist, loading EoD data from file


In [7]:
# prepare currency conversion
currency_conversion = eod_data.loc[currency_ticker, 'Close']
# fix the conversion, it cannot be 0
for i in range(0, currency_conversion.size - 1):
    if currency_conversion.iloc[i] == 0:
        currency_conversion.iloc[i] = currency_conversion.iloc[i + 1]
if currency_conversion.iloc[-1] == 0:
    currency_conversion.iloc[-1] = currency_conversion.iloc[-2]

/Users/peterk/.pyenv/versions/3.7.1/envs/finance-3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/peterk/.pyenv/versions/3.7.1/envs/finance-3.7.1/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# add extra columns
for c in ['Quantity', 'Cost', 'Commission', 'Cash', 'Dividends Value',   # USD
          'Cost CY', 'Commission CY', 'Cash CY', 'Dividends Value CY']:  # currency
    eod_data[c] = 0
if logs > 1: eod_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8001 entries, (IBM, 2015-01-05 00:00:00) to (^IXIC, 2019-07-19 00:00:00)
Data columns (total 12 columns):
Close                 8001 non-null float64
Dividends             8001 non-null float64
Stock Splits          8001 non-null int64
Quantity              8001 non-null int64
Cost                  8001 non-null int64
Commission            8001 non-null int64
Cash                  8001 non-null int64
Dividends Value       8001 non-null int64
Cost CY               8001 non-null int64
Commission CY         8001 non-null int64
Cash CY               8001 non-null int64
Dividends Value CY    8001 non-null int64
dtypes: float64(2), int64(10)
memory usage: 1.1+ MB


In [9]:
# add transactions to eod_data
transactions.set_index(['symbol', 'date'], inplace=True)
for idx, tx in transactions.iterrows():
    symbol, tx_date = idx
    p = eod_data.at[idx, 'Close'] if pd.isna(tx['price']) else tx['price']  # price if set otherwise close value
    q = (-1 if tx['tx'] == 'SELL' else 1) * tx['quantity']
    cy_rate = currency_conversion.at[tx_date]
    eod_data.at[idx, 'Quantity'] += q
    eod_data.at[idx, 'Cost'] += (q * p)
    eod_data.at[idx, 'Cost CY'] += (q * p) * cy_rate
    eod_data.at[idx, 'Commission'] += tx['commission']
    eod_data.at[idx, 'Commission CY'] += tx['commission'] * cy_rate
    eod_data.at[idx, 'Cash'] += tx['cash']
    eod_data.at[idx, 'Cash CY'] += tx['cash'] * cy_rate
    eod_data.at[idx, 'Dividends Value'] += (q * eod_data.at[idx, 'Dividends'])
    eod_data.at[idx, 'Dividends Value CY'] += (q * eod_data.at[idx, 'Dividends']) * cy_rate

In [10]:
# apply cumulative sum to all tickers
def get_cumulative(tickers):

    def get_data(ticker):
        if logs > 1: print(f'Processing {ticker}')
        data = eod_data.loc[ticker][['Close']]
        data['Holdings'] = eod_data.loc[ticker, 'Quantity'].cumsum()
        data['Cost'] = eod_data.loc[ticker, 'Cost'].cumsum()
        data['Cost CY'] = eod_data.loc[ticker, 'Cost CY'].cumsum()
        data['Market Value'] = data['Holdings'] * eod_data.loc[ticker, 'Close']  # not cumulative
        data['Market Value CY'] = data['Market Value'] * currency_conversion  # not cumulative
        data['Dividends'] = eod_data.loc[ticker, 'Dividends Value'].cumsum()
        data['Dividends CY'] = eod_data.loc[ticker, 'Dividends Value CY'].cumsum()
        data['Commission'] = eod_data.loc[ticker, 'Commission'].cumsum()
        data['Commission CY'] = eod_data.loc[ticker, 'Commission CY'].cumsum()
        data['Cash'] = eod_data.loc[ticker, 'Cash'].cumsum()
        data['Cash CY'] = eod_data.loc[ticker, 'Cash CY'].cumsum()
        return data

    datas = list(map(get_data, tickers))
    return pd.concat(datas, keys=tickers, names=['symbol', 'date'], sort=True)


all_data = get_cumulative(tickers + [currency_ticker] + benchmark_tickers)
del(eod_data)  # clean up the eod_data

Processing IBM
Processing MSFT
Processing TSLA
Processing USDGBP=X
Processing ^GSPC
Processing ^DJI
Processing ^IXIC


In [11]:
portfolio_data = pd.DataFrame({'Balance': 0, 'Balance CY': 0, 'Return': 0},
                              index=all_data.loc[benchmark_ticker].index)
for ticker in tickers + [currency_ticker]:
    portfolio_data['Balance'] += (all_data.loc[ticker, 'Market Value']
                                  + all_data.loc[ticker, 'Cash']
                                  + all_data.loc[ticker, 'Dividends'] 
                                  - all_data.loc[ticker, 'Commission']
                                  - all_data.loc[ticker, 'Cost'])
    portfolio_data['Balance CY'] += (all_data.loc[ticker, 'Market Value CY']
                                    + all_data.loc[ticker, 'Cash CY']
                                    + all_data.loc[ticker, 'Dividends CY']
                                    - all_data.loc[ticker, 'Commission CY']
                                    - all_data.loc[ticker, 'Cost CY'])

# Plots and Diagrams

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [13]:
# Configurations ###
from_date = datetime(2015, 1, 1).replace(tzinfo=pytz.UTC)
to_date = end_date

In [14]:
# Adjust dates for plots and diagrams
if from_date < start_date:
    from_date = start_date
    print('Start date for plotting has been adjusted')
if to_date > end_date:
    to_date = end_date
    print('End date for plotting has been adjusted')

Start date for plotting has been adjusted


In [15]:
# graph data holds plot specific data, which might be calculated on a different date range
graph_data = (portfolio_data.loc[from_date:to_date])[['Balance', 'Balance CY', 'Return']]
balance_base = graph_data.iloc[0]['Balance']
graph_data['Balance %'] = graph_data.apply(lambda x: (x['Balance'] - balance_base)/x['Balance'], axis=1)
balance_base_cy = graph_data.iloc[0]['Balance CY']
graph_data['Balance CY %'] = graph_data.apply(lambda x: (x['Balance CY'] - balance_base_cy)/x['Balance CY'], axis=1)
for t in benchmark_tickers:
    graph_data[t] = all_data.loc[t]['Market Value']
    benchmark_base = graph_data.iloc[0][t]
    graph_data[t + ' %'] = graph_data.apply(lambda x: (x[t] - benchmark_base)/x[t], axis=1)

In [16]:
trace1 = go.Scatter(
    x = graph_data.index,
    y = graph_data['Balance %'],
    name = 'Portfolio',
    line=dict(width=1))
trace2 = go.Scatter(
    x = graph_data.index,
    y = graph_data[f'Balance CY %'],
    name = f'Portfolio {currency_ticker}',
    line=dict(width=1))
data = [trace1, trace2]
for t in benchmark_tickers:
    data.append(
        go.Scatter(
            x = graph_data.index,
            y = graph_data[t + ' %'],
            name = t,
            line=dict(width=1))
    )
layout = go.Layout(legend=dict(x=0,y=1,orientation='h'),
    yaxis=dict(tickformat="%", side="right"),
    xaxis=dict(tickformat="%b'%y"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Portfolio Statistics

In [17]:
returns = portfolio_data['Balance'].pct_change()
mean_return = returns.mean()
return_stdev = returns.std()
annualised_return = round(mean_return * 252,2)
annualised_stdev = round(return_stdev * np.sqrt(252),2)
print(f'Annualised mean return = {annualised_return}\nAnnualised volatility = {annualised_stdev}')

Annualised mean return = 0.06
Annualised volatility = 0.16
